In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import confusion_matrix, f1_score
import requests

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## 1) Aprendizagem supervisionada

Para o modelo de aprendizagem supervisionada, tomaremos como exemplo o dataset de notícias do kernel "Getting Started with NLP". 

O objetivo do desafio é classificar notícias entre fake news e notícias reais. Temos os rótulos de nossas observações e os rótulos são binários: ou uma notícia é verdadeira, ou ela é falsa. Trata-se, pois, de um caso de aprendizagem supervisionada para classificação.

Estão descartadas, de imediato, todas as métricas de regressão como "Mean Squared Error", "Mean Absolute Error" e "Max Erro". Trabalharemos aqui com:
* Uma **matriz de confusão**, com os valores de verdadeiros positivos, verdadeiros negativos, falsos positivos e falsos negativos
* Um **relatório de classificação** (classification_report) do scikit-learn, que já traz consigo as métricas típicas de problemas de classificação binária, a saber, **acurácia, precisão e recall** (além da métrica composta **f1_score**, que pondera precisão e recall).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import nltk
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Carregar datasets

sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv", index_col='id')
train = pd.read_csv("../input/nlp-getting-started/train.csv", index_col='id')

In [ ]:
# Retirar pontuação

for col in train.columns:
    if not train[col].dtype == 'int':
        train[col] = train[col].str.lower().replace(r'[^\w\s]','',regex=True)

# Retirar números

train['keyword'] = train.keyword.str.replace(r'\d','',regex=True)
train['location'] = train.location.str.replace(r'\d','',regex=True)

# Remover acentos
train['text'] = train.text.apply(unidecode)

# Remover stopwords

def remover_stopwords(frase):
    
    stop_words = set(stopwords.words('english')) 
    tokens = word_tokenize(frase) 
    frase_filtrada = [p for p in tokens if not p in stop_words] 
    return ' '.join(frase_filtrada)

train['text'] = train.text.apply(remover_stopwords)

lemmatizer = WordNetLemmatizer()

train = train.dropna(subset=['keyword'])
train.keyword = train.keyword.apply(lemmatizer.lemmatize)
train.location.fillna('unknown',inplace=True)

In [ ]:
X = train.drop(columns='target')
y = train.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)

In [ ]:
## Criamos uma pipeline para classificação

vectorizer = CountVectorizer()
classifier = RandomForestClassifier()

pipe = Pipeline([('vectorizer',vectorizer),
                ('clf',classifier)])

pipe.fit(X_train.text, y_train)

In [ ]:
## Invocamos a pipeline com as métricas escolhidas, classification report e matriz de confusão

y_pred = pipe.predict(X_val.text)

print(classification_report(y_val, y_pred), confusion_matrix(y_val, y_pred))

## 2) Aprendizagem não supervisionada

Desta vez, utilizaremos o conjunto de dados "New York City Airbnb Open Data" disponibilizado pelo usuário Dgomonov.

A ideia aqui é utilizar os dados sobre as acomodações - incluindo as descrições feitas pelos anfitriões - para agrupar em tipos mais detalhados de acomodação que podem corresponder a perfis diferentes de usuário. Parte-se da hipótese de que algumas pessoas procuram acomodações aconchegantes e tranquilas, ainda que distantes de pontos turísticos, enquanto outras se esbaldam em uma acomodação agitada e caótica. 

Como não conseguimos prever de antemão quais serão essas categorias, trata-se de um caso de **aprendizagem não supervisionada**.

### Métricas

Adjusted Rand Index, Mutual information based scores, Homogeneity, completness e V-measure não poderão ser utilizadas nesse caso, pois não temos uma "verdade básica" (um conjunto de rótulos reais) para utilizar como parâmetro de comparação.

Por outro lado, temos à nossa disposição:
* **Coeficiente de silhueta**: medida da qualidade da diferenciação dos clusters. Vai de -1 (agrupamento incorreto) a +1 (agrupamento denso). Scores em torno de 0 indicam sobreposição;
* **Índice Clazinski-Harabasz**: mede a qualidade dos clusters em termos da dispersão intra versus inter cluster. Quanto maior a nota, melhores os grupos;
* **Índice Davies-Bouldin**: similaridade, tida como uma relação entre a distância entre os clusters e o seu tamanho. Quanto menor o valor, mais nítida a separação entre clusters.

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [ ]:
# Carregar dataset

airbnb = pd.read_csv("../input/new-york-city-airbnb-open-data/AB_NYC_2019.csv", index_col='id')

airbnb.head()

In [ ]:
## !Utilizar get dummies nos bairros e no tipo de quarto

## Remover colunas de medidas que não são de nosso interesse, como o nome do anfitrião e os dados sobre críticas.

to_drop = ['host_id','host_name','number_of_reviews','last_review','reviews_per_month','calculated_host_listings_count']

airbnb = airbnb.drop(columns=to_drop)

## Também adequamos o tipo de algumas variáveis

airbnb[['neighbourhood_group','neighbourhood','room_type']] = airbnb[['neighbourhood_group','neighbourhood','room_type']].astype('category')

airbnb.info()

In [ ]:
get_categorical = FunctionTransformer(lambda x: pd.get_dummies(x[['neighbourhood_group','neighbourhood','room_type']], drop_first=True))

get_numerical = FunctionTransformer(lambda x:x[['latitude','longitude','price','minimum_nights','availability_365']])
                                      
scaler = StandardScaler()

get_text = FunctionTransformer(lambda x: x['name'])
                                      
vectorizer = HashingVectorizer()

tratar_e_combinar = FeatureUnion(transformer_list=
                    [('categoricas',get_categorical),
                    ('numericas',Pipeline([
                        ('retirar',get_numerical),
                        ('scaler',scaler)])),
                     ('texto',Pipeline([
                         ('retirar',get_text),
                         ('vectorizer',vectorizer)
                     ]))])

kmeans = KMeans(n_clusters=5, random_state=2)

pipe = Pipeline([('tratamento',tratar_e_combinar),
                 ('modelo',kmeans)])

X_train, X_test = train_test_split(airbnb, test_size=0.8)

In [ ]:
### O algoritmo não funcionou :(. Acredito que a vetorização do texto tenha ficado pesada pela falta de tratamento.

X = tratar_e_combinar.fit_transform(X_train)
SpectralClustering().fit(X.toarray())

## 

## 3) Reinforcement Learning

No reinforcement learning não há métricas pré-definidas: a eficácia do nosso algoritmo deve ser medida pelo objetivo que determinados para um ator em um dado ambiente, conforme suas regras.

In [ ]:
# Carregar dataset

OptimalPolicy_angletol45 = pd.read_csv("../input/-reinforcement-learning-from-scratch-in-python/OptimalPolicy_angletol45.csv")

In [ ]:
OptimalPolicy_angletol45.info()